In [22]:
import nltk
import pandas as pd
import torch
from torchtext.datasets import AG_NEWS
from torch.utils.data.dataset import random_split

# Train/Test Split

In [23]:
train_iter, test_iter = AG_NEWS()
next(train_iter)

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

In [24]:
train_dataset = list(train_iter)
test_dataset = list(test_iter)

test_dataset[0]

(3,
 "Fears for T N pension after talks Unions representing workers at Turner   Newall say they are 'disappointed' after talks with stricken parent firm Federal Mogul.")

# TOKENIZATION

In [25]:
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab

In [26]:
tokenizer = get_tokenizer('basic_english')
train_iter = AG_NEWS(split='train')
counter = Counter()

for (label, line) in train_iter:
  counter.update(tokenizer(line))
vocab = Vocab(counter, min_freq=1)

In [27]:
print(f"information dominance: {[vocab[token] for token in ['information', 'dominance']]}")

information dominance: [657, 4416]


In [28]:
text_pipeline = lambda x: [vocab[token] for token in tokenizer(x)]
label_pipeline = lambda x: int(x) - 1

In [29]:
text_pipeline('torch is an example of information dominance'), label_pipeline('5')

([12703, 22, 31, 5298, 7, 657, 4416], 4)

# Data Batch and Iterator (DataLoader)

In [30]:
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

In [31]:
train_iter = AG_NEWS(split='train')
dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)

In [32]:
for i, (labels, text, offsets) in enumerate(dataloader):
    print(f'labels: {labels}')
    print(f'text: {text.size(0)}')
    print(f'offset: {offsets}')
    break

labels: tensor([2, 2, 2, 2, 2, 2, 2, 2])
text: 338
offset: tensor([  0,  29,  71, 111, 151, 194, 242, 289])


# Model Definition

In [33]:
from torch import nn

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)


In [34]:
train_iter = AG_NEWS(split='train')
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)
model

TextClassificationModel(
  (embedding): EmbeddingBag(95812, 64, mode=mean)
  (fc): Linear(in_features=64, out_features=4, bias=True)
)

# Train and Evaluate Functions

In [35]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predited_label = model(text, offsets)
        loss = criterion(predited_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predited_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

In [36]:
def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predited_label = model(text, offsets)
            loss = criterion(predited_label, label)
            total_acc += (predited_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

# Run the Models

## 1.  Define Hyperparameters

In [37]:
# Hyperparameters
EPOCHS = 10 # epoch
LR = 0.05  # learning rate
BATCH_SIZE = 64 # batch size for training

## 2.  Define Define Loss Function, Optimizer, and Learning Rate

In [38]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None

In [39]:
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = random_split(train_dataset, [num_train, len(train_dataset) - num_train])
split_train_

## 3. Define the DataLoader for train, test, split
#### Before sending to the model, collate_fn function works on a batch of samples generated from DataLoader. The input to collate_fn is a batch of data with the batch size in DataLoader, and collate_fn processes them according to the data processing pipelines declared previouly. Pay attention here and make sure that collate_fn is declared as a top level def. This ensures that the function is available in each worker.

In [40]:
train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

In [41]:
## 4.  Training Loops

In [42]:
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

| epoch   1 |   500/ 1782 batches | accuracy    0.253
| epoch   1 |  1000/ 1782 batches | accuracy    0.294
| epoch   1 |  1500/ 1782 batches | accuracy    0.333
-----------------------------------------------------------
| end of epoch   1 | time: 18.95s | valid accuracy    0.379 
-----------------------------------------------------------
| epoch   2 |   500/ 1782 batches | accuracy    0.379
| epoch   2 |  1000/ 1782 batches | accuracy    0.406
| epoch   2 |  1500/ 1782 batches | accuracy    0.442
-----------------------------------------------------------
| end of epoch   2 | time: 19.08s | valid accuracy    0.474 
-----------------------------------------------------------
| epoch   3 |   500/ 1782 batches | accuracy    0.470
| epoch   3 |  1000/ 1782 batches | accuracy    0.484
| epoch   3 |  1500/ 1782 batches | accuracy    0.497
-----------------------------------------------------------
| end of epoch   3 | time: 18.89s | valid accuracy    0.527 
-------------------------------

## 4.  Evaluate the Model with test dataset

In [43]:
print('Checking the results of test dataset.')
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.3f}'.format(accu_test))

Checking the results of test dataset.
test accuracy    0.735


## 5. Test on a random News

In [44]:
ag_news_label = {1: "World",
                 2: "Sports",
                 3: "Business",
                 4: "Sci/Tec"}

def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text))
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item() + 1

ex_text_str = "MEMPHIS, Tenn. – Four days ago, Jon Rahm was \
    enduring the season’s worst weather conditions on Sunday at The \
    Open on his way to a closing 75 at Royal Portrush, which \
    considering the wind and the rain was a respectable showing. \
    Thursday’s first round at the WGC-FedEx St. Jude Invitational \
    was another story. With temperatures in the mid-80s and hardly any \
    wind, the Spaniard was 13 strokes better in a flawless round. \
    Thanks to his best putting performance on the PGA Tour, Rahm \
    finished with an 8-under 62 for a three-stroke lead, which \
    was even more impressive considering he’d never played the \
    front nine at TPC Southwind."

model = model.to("cpu")

print("This is a %s news" %ag_news_label[predict(ex_text_str, text_pipeline)])

This is a Sports news
